# Advanced Practice: The `zip()` Function

These exercises deepen your understanding of `zip()` and common patterns built on top of it:

- strict length checking and safe unzipping,
- pairwise & sliding windows,
- matrix transposition (strict/truncate/fill),
- vector math & key-aligned joins,
- interleaving and parallel sorting with unzip.

👉 **Instructions**
- Implement where marked `# YOUR CODE HERE`.
- Do **not** change test cells.
- Only the standard library is needed.


In [1]:
from typing import Iterable, Iterator, Tuple, List, Sequence, Any, Optional
from itertools import islice, zip_longest


## Problem 1 — `pairwise(iterable)` (adjacent pairs)

Return an iterator of adjacent pairs `(x[i], x[i+1])`.

Use a single pass with `zip(it, islice(it, 1, None))`.

Edge cases: empty / length 1 should yield nothing.

In [2]:
def pairwise(iterable: Iterable[Any]) -> Iterator[Tuple[Any, Any]]:
    """Yield (a,b) adjacent pairs from iterable.

    Examples
    --------
    >>> list(pairwise([1,2,3,4]))
    [(1, 2), (2, 3), (3, 4)]
    >>> list(pairwise([]))
    []
    """
    it = iter(iterable)
    return zip(it, islice(iter(iterable), 1, None))


In [3]:
# Tests — do not modify
assert list(pairwise([1,2,3,4])) == [(1,2),(2,3),(3,4)]
assert list(pairwise([42])) == []
assert list(pairwise([])) == []
print("✅ Problem 1 tests passed.")


✅ Problem 1 tests passed.


## Problem 2 — `sliding_window(iterable, n)`

Return an iterator of overlapping windows of size `n` as tuples, e.g. for `n=3`:
`[(x0,x1,x2), (x1,x2,x3), ...]`.

Use the canonical `zip(islice(...), islice(...), ...)` approach. For `n <= 0`, return an empty iterator.

In [4]:
def sliding_window(iterable: Iterable[Any], n: int) -> Iterator[Tuple[Any, ...]]:
    """Yield n-length sliding windows over iterable.

    >>> list(sliding_window([1,2,3,4], 3))
    [(1,2,3), (2,3,4)]
    """
    if n <= 0:
        return iter(())
    it = iter(iterable)
    iterators = [islice(iter(iterable), i, None) for i in range(n)]
    return zip(*iterators)


In [5]:
# Tests — do not modify
assert list(sliding_window([1,2,3,4], 3)) == [(1,2,3),(2,3,4)]
assert list(sliding_window([1,2], 3)) == []
assert list(sliding_window([1,2,3], 1)) == [(1,),(2,),(3,)]
assert list(sliding_window([1,2,3], 0)) == []
print("✅ Problem 2 tests passed.")


✅ Problem 2 tests passed.


## Problem 3 — Safe `unzip(pairs)`

Implement `unzip(pairs)` that returns two **lists** `(A,B)` from a sequence of `(a,b)`.

Edge case: empty input → return `([], [])` (avoid `ValueError` from `zip(*)`).

In [6]:
def unzip(pairs: Iterable[Tuple[Any, Any]]) -> Tuple[List[Any], List[Any]]:
    """Return two lists from an iterable of pairs. Empty-safe.

    >>> unzip([(1,'a'), (2,'b')])
    ([1,2], ['a','b'])
    """
    pairs = list(pairs)
    if not pairs:
        return ([], [])
    a, b = zip(*pairs)
    return [*a], [*b]


In [7]:
# Tests — do not modify
A, B = unzip([(1,'a'), (2,'b'), (3,'c')])
assert A == [1,2,3] and B == ['a','b','c']
assert unzip([]) == ([], [])
print("✅ Problem 3 tests passed.")


✅ Problem 3 tests passed.


## Problem 4 — Matrix transpose with modes

Implement `transpose(matrix, mode='strict', fillvalue=None)` where `matrix` is an iterable of rows (iterables):

- `mode='strict'`: like `zip(*rows, strict=True)` — raise `ValueError` if row lengths differ.
- `mode='truncate'`: like plain `zip(*rows)` — stop at the shortest row.
- `mode='fill'`: like `zip_longest(*rows, fillvalue=fillvalue)` — pad to the longest row.

Return a **list of lists**.

In [8]:
def transpose(matrix: Iterable[Iterable[Any]], mode: str = 'strict', fillvalue: Any = None) -> List[List[Any]]:
    rows = [list(r) for r in matrix]
    if not rows:
        return []
    if mode == 'strict':
        # emulate zip(*rows, strict=True) behavior (3.10+ has strict param on zip)
        length = len(rows[0])
        if any(len(r) != length for r in rows):
            raise ValueError('row lengths differ in strict mode')
        return [list(t) for t in zip(*rows)]
    elif mode == 'truncate':
        return [list(t) for t in zip(*rows)]
    elif mode == 'fill':
        return [list(t) for t in zip_longest(*rows, fillvalue=fillvalue)]
    else:
        raise ValueError("mode must be one of: 'strict', 'truncate', 'fill'")


In [9]:
# Tests — do not modify
m = [[1,2,3],[4,5,6]]
assert transpose(m, 'strict') == [[1,4],[2,5],[3,6]]
assert transpose([[1,2],[3,4,5]], 'truncate') == [[1,3],[2,4]]
assert transpose([[1,2],[3]], 'fill', fillvalue=0) == [[1,3],[2,0]]
try:
    transpose([[1],[2,3]], 'strict')
    raise AssertionError('expected ValueError')
except ValueError:
    pass
print("✅ Problem 4 tests passed.")


✅ Problem 4 tests passed.


## Problem 5 — Dot product with strict length checking

Implement `dot(u, v)` for numeric sequences. Use `zip(u, v, strict=True)` logic (raise if lengths differ). Return a number.

In [10]:
def dot(u: Sequence[float], v: Sequence[float]) -> float:
    """Return the dot product of u and v.
    Raises ValueError if lengths differ.
    """
    if len(u) != len(v):
        raise ValueError('length mismatch')
    return sum(a*b for a, b in zip(u, v))


In [11]:
# Tests — do not modify
assert dot([1,2,3], [4,5,6]) == 32
try:
    dot([1,2], [3])
    raise AssertionError('expected ValueError')
except ValueError:
    pass
print("✅ Problem 5 tests passed.")


✅ Problem 5 tests passed.


## Problem 6 — Align by keys (outer join style)

Given `keys` and multiple dictionaries `*series` mapping `key -> value`, produce a list of tuples:

`[(key, s1.get(key, fill), s2.get(key, fill), ...)]` in the order of `keys`.

Implement `align_by_key(keys, *series, fillvalue=None)`.

Hints: For each key, `zip` the series lookups to build the tuple `(key, *values)`.

In [12]:
def align_by_key(keys: Iterable[Any], *series: dict, fillvalue: Any = None) -> List[Tuple[Any, ...]]:
    """Align values from multiple dicts according to keys; missing → fillvalue.

    Examples
    --------
    >>> keys = ['A','B','C']
    >>> s1 = {'A':1,'C':3}
    >>> s2 = {'B':20,'C':30}
    >>> align_by_key(keys, s1, s2, fillvalue=0)
    [('A', 1, 0), ('B', 0, 20), ('C', 3, 30)]
    """
    out: List[Tuple[Any, ...]] = []
    for k in keys:
        vals = tuple(s.get(k, fillvalue) for s in series)
        out.append((k, *vals))
    return out


In [13]:
# Tests — do not modify
keys = ['A','B','C']
s1 = {'A': 1, 'C': 3}
s2 = {'B': 20, 'C': 30}
s3 = {'A': 100}
res = align_by_key(keys, s1, s2, s3, fillvalue=0)
assert res == [('A', 1, 0, 100), ('B', 0, 20, 0), ('C', 3, 30, 0)]
print("✅ Problem 6 tests passed.")


✅ Problem 6 tests passed.


## Problem 7 — Parallel sort with unzip (names by scores)

Implement `top_k(names, scores, k)` that returns the top-`k` `(name, score)` by **descending score**, breaking ties by **name ascending**.

Strategy:
- `zip(names, scores)` to pair,
- `sorted(..., key=(−score, name))`,
- slice first `k`.

Return a **list of tuples**.

In [14]:
def top_k(names: Sequence[str], scores: Sequence[float], k: int) -> List[Tuple[str, float]]:
    if len(names) != len(scores):
        raise ValueError('length mismatch')
    if k <= 0:
        return []
    pairs = list(zip(names, scores))
    pairs.sort(key=lambda p: (-p[1], p[0]))
    return pairs[:k]


In [15]:
# Tests — do not modify
names = ['Ann','Bob','Ava','Ben']
scores = [90, 95, 95, 88]
assert top_k(names, scores, 3) == [('Ava',95), ('Bob',95), ('Ann',90)]
assert top_k(names, scores, 0) == []
try:
    top_k(['x'], [], 1)
    raise AssertionError('expected ValueError')
except ValueError:
    pass
print("✅ Problem 7 tests passed.")


✅ Problem 7 tests passed.


## Problem 8 — Interleave two iterables

Implement `interleave(a, b)` that yields items as `a0, b0, a1, b1, ...`.

- If lengths differ, append the remainder of the longer.
- Implement with `zip_longest` and skip `None` sentinels safely (use a unique sentinel).

Return a **list**.

In [16]:
def interleave(a: Iterable[Any], b: Iterable[Any]) -> List[Any]:
    """Return [a0,b0,a1,b1,...]; if one is longer, append its tail.
    Uses a sentinel to avoid colliding with user data.
    """
    _MISSING = object()
    out: List[Any] = []
    for x, y in zip_longest(a, b, fillvalue=_MISSING):
        if x is not _MISSING:
            out.append(x)
        if y is not _MISSING:
            out.append(y)
    return out


In [17]:
# Tests — do not modify
assert interleave([1,2,3], ['a','b','c']) == [1,'a',2,'b',3,'c']
assert interleave([1,2,3], ['a']) == [1,'a',2,3]
assert interleave([], [10,20]) == [10,20]
print("✅ Problem 8 tests passed.")


✅ Problem 8 tests passed.
